### Введение

Целью данной работы является исследование применимости алгоритмов ускорения умножения матриц для аппраратной реализации в нейростях. Был выбран алгоритм [Mithral](https://arxiv.org/pdf/2106.10860.pdf)

### 1.  Постановка задачи

$A: l \times m \rightarrow \mathbb{R}$ - матрица входных данных.

$B: m \times n \rightarrow \mathbb{R}$ - матрица с весами нейросети. Её значение будем считать неизменным.

$C = A \times B $ - результат (или часть результатата) применения слоя нейросести.

<!-- $C' = C + \epsilon, |\epsilon| \rightarrow 0$ -->

$\overline{C} = f_B(A)$ - приблеженный результат применения, полученный с использование выбранного алгоритма.

$|\overline{C} - C| < \epsilon $ -- данное определение ошибки не является конечным и не определяет испрользование нормы $L_1$.

$ O(f_B) < O(n^3) $ -- выбранный алгоритм должен быть быстрее непосредственного перемножения матриц.


### 1. Общие сведения

Процесс управляется кодом в файле [amm_main.py](https://github.com/HornedHeck/bolt/blob/master/experiments/python/amm_main.py) в функции [_main](https://github.com/HornedHeck/bolt/blob/99dfffb1e2c5c43cfd1d7746f0dee6b350981bbd/experiments/python/amm_main.py#L317). Её можно разделить на следующие этапы:

1. Инициализация [amm_main.py 321-349](https://github.com/HornedHeck/bolt/blob/99dfffb1e2c5c43cfd1d7746f0dee6b350981bbd/experiments/python/amm_main.py#L321-L349)
2. Обучение [amm_main.py 351-380](https://github.com/HornedHeck/bolt/blob/99dfffb1e2c5c43cfd1d7746f0dee6b350981bbd/experiments/python/amm_main.py#L351-L380)
3. Валидация и вывод метрик [amm_main.py 386-401](https://github.com/HornedHeck/bolt/blob/99dfffb1e2c5c43cfd1d7746f0dee6b350981bbd/experiments/python/amm_main.py#L387-L401)

Далее будет рассматриваться только этап валидации а в частности функция `_eval_amm` [amm_main.py 264](https://github.com/HornedHeck/bolt/blob/99dfffb1e2c5c43cfd1d7746f0dee6b350981bbd/experiments/python/amm_main.py#L265), которая содержит все этапы выбранного алгоритма.

Сам алгоритм предполагает следующие действия:
1. Кодирование матрицы $B$. Если матрица идентична для нескольких операций умножения, то данное действие можно провести заранее и не включать в длительность выполнения. Учитывая применение к нейросетям ($B$ -- матрица весов или фильтр-свертка) будем использовать именно такой подход.
2. Кодирование матрицы $A$.
3. Вычисление результата $\overline{C}$.


### 2. Описание переменных

1. `MithralEncoder.ncodebooks` -- параметр, регулирует кразмеры матриц для приближения.
2. `MithralEncoder.ncentroids` -- формально также является параметром, регулирующим точность приближения, однако перманентно уставнолен в значение 16.
3. `MithralEncoder.all_centroids` -- массив размера `ncodebooks` $\times$ `ncentroids` $\times m$ с данными о кодируемых значениях, получен в результате обучения.
4. `Q`, `W` -- в некоторых частях программы так обозначена матрица $B$.
5. `X` -- в некоторых частях программы так обозначена матрица $A$.
6. `Y` -- в некоторых частях программы так обозначена матрица $\overline{C}$.
7. `MithralEncoder.splits_lists` -- список пороговых значений, используемых при кодировании матрицы А. Получен в результате обучения. Имеет размер $ncodebooks \times nplits$.
8. nsplits -- параметр, не более 4, по умолчанию 4.
9. MithralEncoder.offsets -- массив элементов вида: $0, ncentroids, 2 \cdot ncentroids, ...$ размера `ncdoebooks`
10. MithralEncoder.upcast_every -- параметр, решулирующий порядок преобразований при получени ответа. Должен быть делителем `ncodebooks`.

### 3. Кодирование матрицы $B$
##### Исходный код:
* [MithralMatmul.set_B](https://github.com/HornedHeck/bolt/blob/99dfffb1e2c5c43cfd1d7746f0dee6b350981bbd/experiments/python/vq_amm.py#L311)
* [MithralEncoder.encode_Q](https://github.com/HornedHeck/bolt/blob/99dfffb1e2c5c43cfd1d7746f0dee6b350981bbd/experiments/python/vquantizers.py#L614)
* [_mithral_quantize_luts](https://github.com/HornedHeck/bolt/blob/99dfffb1e2c5c43cfd1d7746f0dee6b350981bbd/experiments/python/vquantizers.py#L481)
* [mithral_lut](https://github.com/HornedHeck/bolt/blob/99dfffb1e2c5c43cfd1d7746f0dee6b350981bbd/experiments/python/clusterize.py#L1658)

##### Действия
1. Транспонирование матрицы B [vq_amm.py 312:64](https://github.com/HornedHeck/bolt/blob/99dfffb1e2c5c43cfd1d7746f0dee6b350981bbd/experiments/python/vq_amm.py#L312)
2. Создание и заполнение массива `luts` размером $n \times ncodebooks \times ncentroids$ [vquantizers.py 616-618](https://github.com/HornedHeck/bolt/blob/99dfffb1e2c5c43cfd1d7746f0dee6b350981bbd/experiments/python/vquantizers.py#L616-L618)
	1. Умножение каждого вектора массива `all_centriods` на строку `q` матрицы $B$. [clusterize.py 1660:12](https://github.com/HornedHeck/bolt/blob/99dfffb1e2c5c43cfd1d7746f0dee6b350981bbd/experiments/python/clusterize.py#L1660)
	2. Полученный массив складывается вдоль последней оси (получается матрица размера `ncodebooks` $\times$ `ncentroids`). Из этих матриц формируется массив `luts`. [clusterize.py 1660:32](https://github.com/HornedHeck/bolt/blob/99dfffb1e2c5c43cfd1d7746f0dee6b350981bbd/experiments/python/clusterize.py#L1660)
3. Далее выполняется смещение и маштабирование переменной `luts`, поскольу параметр `MithralEncoder.quantize_lut` всегда в значении `true`. [vquantizers.py 619-620](https://github.com/HornedHeck/bolt/blob/99dfffb1e2c5c43cfd1d7746f0dee6b350981bbd/experiments/python/vquantizers.py#L619-L620)
	1. Вычисляются минимальные и максимальные значения массива `luts` вдоль первой и последней оси. Т.е. вычисляются значения в каждом одномерном массиве и заносятся в матрицу, затем в этой матрице выбираются минимальные\максиммальные значение по столбцам. В рузельтате получаем вектора `mins`, `maxs` размером `ncodebooks`. [vquantizers.py 549-550](https://github.com/HornedHeck/bolt/blob/99dfffb1e2c5c43cfd1d7746f0dee6b350981bbd/experiments/python/vquantizers.py#L549-L550)
	2. Вычисляется покомпонетная разница векторов `maxs` и `mins` и заносится в массив `gaps`. [vquantizers.py 552:5](https://github.com/HornedHeck/bolt/blob/99dfffb1e2c5c43cfd1d7746f0dee6b350981bbd/experiments/python/vquantizers.py#L552)
	3. Вычисляется максимальная разница `gap`. [vquantizers.py 554:5](https://github.com/HornedHeck/bolt/blob/99dfffb1e2c5c43cfd1d7746f0dee6b350981bbd/experiments/python/vquantizers.py#L554)
	4. Вычисляется маштаб, который будет применен к данным:
	$$ scale = 2^{-1 - [log_2(gap)]} \cdot (255.5 - 10^{-10}) $$
	Умножение производится для максимального приблежения максимального закодированного значения к 255 (по прим. разр.). [vquantizers.py 556-558](https://github.com/HornedHeck/bolt/blob/99dfffb1e2c5c43cfd1d7746f0dee6b350981bbd/experiments/python/vquantizers.py#L556-L558)
	5. В качестве смещения используются соответствующие ээлементы вектора `mins` в виде переменной `offsets`. [vquantizers.py 562:5](https://github.com/HornedHeck/bolt/blob/99dfffb1e2c5c43cfd1d7746f0dee6b350981bbd/experiments/python/vquantizers.py#L562)
	6. Смещение и маштабирование массива `luts`. Результат помещается в переменную `luts_quantized` [vquantizers.py 563:5](https://github.com/HornedHeck/bolt/blob/99dfffb1e2c5c43cfd1d7746f0dee6b350981bbd/experiments/python/vquantizers.py#L563)
	7. Приведение к целому числу элементов `luts_quantized` по стандартным правилам округления. [vquantizers.py 564:5](https://github.com/HornedHeck/bolt/blob/99dfffb1e2c5c43cfd1d7746f0dee6b350981bbd/experiments/python/vquantizers.py#L564)
	8. В результате получаем [vquantizers.py 572:5](https://github.com/HornedHeck/bolt/blob/99dfffb1e2c5c43cfd1d7746f0dee6b350981bbd/experiments/python/vquantizers.py#L572):
		1. Закодированную матрицу $B$ в переменной `luts`.
		2. Смещение, как сумму элементов вектора `offsets`.
		3. Маштаб.



### 4. Кодирование матрицы $A$
##### Исходный код:
* [MithralEncoder.encode_X](https://github.com/HornedHeck/bolt/blob/99dfffb1e2c5c43cfd1d7746f0dee6b350981bbd/experiments/python/vquantizers.py#L610)
* [MithralMatmul.set_A](https://github.com/HornedHeck/bolt/blob/99dfffb1e2c5c43cfd1d7746f0dee6b350981bbd/experiments/python/vq_amm.py#L47)
* [mithral_encode](https://github.com/HornedHeck/bolt/blob/99dfffb1e2c5c43cfd1d7746f0dee6b350981bbd/experiments/python/clusterize.py#L1649)
* [assignments_from_multisplits](https://github.com/HornedHeck/bolt/blob/99dfffb1e2c5c43cfd1d7746f0dee6b350981bbd/experiments/python/clusterize.py#L1910)

##### Действия
1. Создается матрица `X_enc` закодированных значений размером $l \times$ `ncodebooks`. [clusterize.py 1658:5](https://github.com/HornedHeck/bolt/blob/0c8a6b4feaf6da1a96b24f8896898689cc9d3a14/experiments/python/clusterize.py#L1658)
2. Каждый столбец $j$ матрицы `X_enc` формаируется следующим образом:
	1. Берется `nsplits` разбиений из `splits_list` по индексу `c`. [clusterize.py 1660](https://github.com/HornedHeck/bolt/blob/0c8a6b4feaf6da1a96b24f8896898689cc9d3a14/experiments/python/clusterize.py#L1660)
	2. Вычисляется `nsplits_affecting_group_id` сколько разбиений можно провести с обновлением значений для кодировки. Кол-во вычислется так, чтобы $2^{nsplits_affecting_group_id}$ не превышало размер максимального(последнего) разбиения. [clusterize.py 1925-1926](https://github.com/HornedHeck/bolt/blob/0c8a6b4feaf6da1a96b24f8896898689cc9d3a14/experiments/python/clusterize.py#L1660)
	3. Формируется вектор `group_ids` размера $l$.
	4. Затем, для каждого из первых $min(nsplits, nsplits_affecting_group_id)$ разбиений: [clusterize.py 1933-1943](https://github.com/HornedHeck/bolt/blob/0c8a6b4feaf6da1a96b24f8896898689cc9d3a14/experiments/python/clusterize.py#L1933-L1943)
		1. Вычисляются пороговые значения для каждого элемента из `group_ids` и заностяся в вектор `vals`. [clusterize.py 1935:9](https://github.com/HornedHeck/bolt/blob/0c8a6b4feaf6da1a96b24f8896898689cc9d3a14/experiments/python/clusterize.py#L1935)
		2. На основе выбранного в разбиении столбца `split.dim`, смешения `split.offset`, маштаба `split.scale` и предидущих вычислений создается массив `indicators`. [clusterize.py 1942:9](https://github.com/HornedHeck/bolt/blob/0c8a6b4feaf6da1a96b24f8896898689cc9d3a14/experiments/python/clusterize.py#L1942)
		$$ 		(X_{split.dim} - split.offset) \cdot split.scale < vals 		$$
		3. Данные заносятся в `group_ids`. [clusterize.py 1943:9](https://github.com/HornedHeck/bolt/blob/0c8a6b4feaf6da1a96b24f8896898689cc9d3a14/experiments/python/clusterize.py#L1943)

		$$ group\_ids = (group\_ids \cdot 2) + indicators $$
	5. Если все разбиения выполнены (`nsplits` $<=$ `nsplits_affecting_group_id`), то `group_ids` -- столбец матрицы `X_enc`, иначе продолжаем вычисления. [clusterize.py 1945-1946](https://github.com/HornedHeck/bolt/blob/0c8a6b4feaf6da1a96b24f8896898689cc9d3a14/experiments/python/clusterize.py#L1945-L1946)
	6. Создаем вектор `assignment` (копия `group_idx`). [clusterize.py 1950:5]
	7. Затем, для каждого из оставшихся разбиений: [clusterize.py 1951-1961]
		1. Вычисляются пороговые значения для каждого элемента из `group_ids` и заностяся в вектор `vals`. [clusterize.py 1952:9](https://github.com/HornedHeck/bolt/blob/0c8a6b4feaf6da1a96b24f8896898689cc9d3a14/experiments/python/clusterize.py#L1952)
		2. На основе выбранного в разбиении столбца `split.dim`, смешения `split.offset`, маштаба `split.scale` и предидущих вычислений создается массив `indicators`. [clusterize.py 1959:9](https://github.com/HornedHeck/bolt/blob/0c8a6b4feaf6da1a96b24f8896898689cc9d3a14/experiments/python/clusterize.py#L1959)
		$$ 		(X_{split.dim} - split.offset) \cdot split.scale < vals 		$$
		3. Данные заносятся в `assignment`. [clusterize.py 1960:9](https://github.com/HornedHeck/bolt/blob/0c8a6b4feaf6da1a96b24f8896898689cc9d3a14/experiments/python/clusterize.py#L1960)

		$$ assignment = (assignment \cdot 2) + indicators $$
	8. `assignment` -- столбец матрицы `X_enc`

3. Смещение элементов закодированных путем построчного сложения с вектором `MithralEncoder.offsets`. [vquantizers.py 612:9](https://github.com/HornedHeck/bolt/blob/99dfffb1e2c5c43cfd1d7746f0dee6b350981bbd/experiments/python/vquantizers.py#L612)

### 5. Получение результата
##### Исходный код:
* [MithralEncoder.dists_enc](https://github.com/HornedHeck/bolt/blob/99dfffb1e2c5c43cfd1d7746f0dee6b350981bbd/experiments/python/vquantizers.py#L224)
* [MithralMatmul.\_\_call\_\_](https://github.com/HornedHeck/bolt/blob/99dfffb1e2c5c43cfd1d7746f0dee6b350981bbd/experiments/python/vq_amm.py#L314)

##### Действия
1. Создается матрица `all_dists` размером $n \times l$, которая будет содержать пезультат умножения.[vquantizers.py 232:9](https://github.com/HornedHeck/bolt/blob/99dfffb1e2c5c43cfd1d7746f0dee6b350981bbd/experiments/python/vquantizers.py#L232)
2. Заполняем `all_dists` за счет следующих действий с каждой из $n$ матриц `lut` из `luts`: [vquantizers.py 233-295](https://github.com/HornedHeck/bolt/blob/99dfffb1e2c5c43cfd1d7746f0dee6b350981bbd/experiments/python/vquantizers.py#L233-L295)
    1. Вычисляются начальные данные (`dists`) для результата путем индексирования 1-мерного представления матрицы `lut` значениями из `X_enc`. [vquantizers.py 234-235](https://github.com/HornedHeck/bolt/blob/99dfffb1e2c5c43cfd1d7746f0dee6b350981bbd/experiments/python/vquantizers.py#L234-L235)
    2. матрица `dists` преобразуется в массив размера $l \times \frac{ncodebooks}{MithralEncoder.upcast\_every} \times MithralEncoder.upcast\_every$. [vquantizers.py 239:17](https://github.com/HornedHeck/bolt/blob/99dfffb1e2c5c43cfd1d7746f0dee6b350981bbd/experiments/python/vquantizers.py#L239)
    3. Уменьшается размерность `dists` путем нахождения среднего арифметичексого с использованием целочисленного деления по элементам вдоль последней оси. В результате получаем матрицу размера $l \times \frac{ncodebooks}{MithralEncoder.upcast_every}$. [vquantizers.py 252-254](https://github.com/HornedHeck/bolt/blob/99dfffb1e2c5c43cfd1d7746f0dee6b350981bbd/experiments/python/vquantizers.py#L252-L254)
    4. Ещё одно уменьшение размерности, путем сложения компонентов вдоль последней оси. Размер вектора после этой операции: $l$. [vquantizers.py 255:21](https://github.com/HornedHeck/bolt/blob/99dfffb1e2c5c43cfd1d7746f0dee6b350981bbd/experiments/python/vquantizers.py#L255)
    5. Выполняются следующие арифметические преорбразование с вектором `dists`  [vquantizers.py 287-300](https://github.com/HornedHeck/bolt/blob/99dfffb1e2c5c43cfd1d7746f0dee6b350981bbd/experiments/python/vquantizers.py#L287-L300) :
    $$  dists = dists \cdot MithralEncoder.upcast\_every - [ncodebooks / 4 * \log_2(MithralEncoder.upcast\_every)]
        dists = dists / scale + offset $$
    6. В результате вектор `dists` хранит строку матрицы `all_dists`.
3. Транспонирование матрицы `all_dists`. После этого в ней содержится результат умножения.